## Gradient Tape

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.datasets import mnist
from tensorflow.keras import optimizers

#### A simple Dense class

In [7]:
class VanillaDense:
    
    def __init__(self, input_size, output_size, activation):
        
        self.activation = activation

        w_shape = (input_size, output_size)
        self.W = tf.Variable(tf.random.uniform(w_shape, minval=0, maxval=1e-1))

        b_shape = (output_size, )
        self.b = tf.Variable(tf.zeros(b_shape))

    def __call__(self, inputs):
        
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        
        return [self.W, self.b]

#### A simple Sequential class

In [8]:
class VanillaSequential:
    
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        
        x = inputs
        
        for layer in self.layers:
            x = layer(x)
        
        return x

    @property
    def weights(self):
       
        weights = []
        
        for layer in self.layers:
            weights += layer.weights
        
        return weights

In [10]:
model = VanillaSequential([
    VanillaDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    VanillaDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

#### A batch generator

In [11]:
class BatchGenerator:
    
    def __init__(self, images, labels, batch_size=128):
        
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        
        return images, labels

### Running one training step

In [18]:
def one_training_step(model, images, labels):
    
    with tf.GradientTape() as tape:
        predictions = model(images)
        losses = sparse_categorical_crossentropy(labels, predictions)
        average_loss = tf.reduce_mean(losses)
        
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    
    return average_loss

In [19]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    
    for g, w in zip(gradients, weights):
        
        w.assign_sub(g * learning_rate)

In [20]:
optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    
    optimizer.apply_gradients(zip(gradients, weights))

### The full training loop

In [21]:
def fit(model, images, labels, epochs, batch_size=128):
    
    for epoch_counter in range(epochs):
        
        print(f"Epoch {epoch_counter}")
        
        batch_generator = BatchGenerator(images, labels)
        
        for batch_counter in range(batch_generator.num_batches):
            
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

In [22]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 4.42
loss at batch 100: 2.23
loss at batch 200: 2.20
loss at batch 300: 2.07
loss at batch 400: 2.23
Epoch 1
loss at batch 0: 1.90
loss at batch 100: 1.87
loss at batch 200: 1.83
loss at batch 300: 1.69
loss at batch 400: 1.84
Epoch 2
loss at batch 0: 1.57
loss at batch 100: 1.58
loss at batch 200: 1.51
loss at batch 300: 1.41
loss at batch 400: 1.52
Epoch 3
loss at batch 0: 1.32
loss at batch 100: 1.34
loss at batch 200: 1.25
loss at batch 300: 1.20
loss at batch 400: 1.28
Epoch 4
loss at batch 0: 1.12
loss at batch 100: 1.16
loss at batch 200: 1.05
loss at batch 300: 1.04
loss at batch 400: 1.12
Epoch 5
loss at batch 0: 0.98
loss at batch 100: 1.02
loss at batch 200: 0.91
loss at batch 300: 0.92
loss at batch 400: 1.00
Epoch 6
loss at batch 0: 0.87
loss at batch 100: 0.91
loss at batch 200: 0.81
loss at batch 300: 0.83
loss at batch 400: 0.91
Epoch 7
loss at batch 0: 0.79
loss at batch 100: 0.83
loss at batch 200: 0.73
loss at batch 300: 0.76
loss at batch 40

### Evaluating the model

In [23]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.81
